# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686613


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:44,  3.72s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:11,  2.66s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:50,  1.96s/it]

Rendering models:  20%|██        | 6/30 [00:08<00:33,  1.42s/it]

...

Rendering models:  77%|███████▋  | 23/30 [00:13<00:02,  3.34it/s]

Rendering models:  93%|█████████▎| 28/30 [00:13<00:00,  4.35it/s]

Rendering models:  97%|█████████▋| 29/30 [00:14<00:00,  4.00it/s]

Rendering models: 100%|██████████| 30/30 [00:14<00:00,  3.25it/s]

SkylerDoesSci                         0.000926
not-logged-in-a9247261537fa1da33fe    0.512030
Camelron                              0.000094
mkehrli                               0.000087
shocko61                              0.000120
not-logged-in-53accdb8e3391ff2ce48    0.000150
djswanso                              0.000082
kjkavene                              0.000094
bldelacr                              0.000109
wrxtminecrafter                       0.000217
lsautter                              0.000093
not-logged-in-41e9fe197dc7a2be8424    0.000778
danielrwik                            0.000076
not-logged-in-376bb2d1d25cc05fc9f1    0.000427
not-logged-in-f8c2f88355fba4282271    0.000665
Planetme                              0.000187
not-logged-in-fcfcbc88feb1aa0ed0a7    0.000157
BrowardDaniel                         0.000329
BigTribble                            0.000213
syt18                                 0.000112
not-logged-in-04a9ceae95d36fd70b9a    0.000278
vceccare     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())